In [1]:
import os
import glob
import io
import collections
import pickle

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats

In [2]:
def read_file(filename):
    # Joined path and filename using os.path.join() to make it more robust and platform-independent.
    path = os.path.join(filename)
    # Used read_table() instead of read_csv() as it is faster and more memory-efficient when reading tab-delimited files.
    df = pd.read_table(path, index_col=[0,1])
    return df

def get_failure(df):
    # check if total error is more than 10 kcal/mol and flag faliures
    df['acceptance_total_err'] = np.where(df['error(total)'] > 10, False, True)
    # check if contirubution of restraints in the complex is more 
    # than 2.5 times the median for all replicas and compunds of the condition
    df['acceptance_site_contr'] = np.where(df['ΔG(site,c+o+a+r)'] > 2.5 * df['ΔG(site,c+o+a+r)'].median(), True, False)
    # check which replica is failed
    df['acceptance_final'] = np.where(df['acceptance_site_contr'] & df['acceptance_total_err'], True, False)
    fail = df[df['acceptance_final'] == False]

def write_accept_reject(df, dir_name, out_name):
    # Initialize lists to store cmpnd and rep for acceptance_final as True and False
    cmpnd_list_acceptance_true = []
    rep_list_acceptance_true = []
    cmpnd_list_acceptance_false = []
    rep_list_acceptance_false = []
    
    # Loop through the rows of the dataframe and append cmpnd and rep to the appropriate lists
    for index, row in df.iterrows():
        # change the rep to match the directory names in repo
        rep_name = index[1]
        new_rep_name = rep_name[3] + '-' + rep_name
        
        cmpnd_name = index[0]
        if cmpnd_name == 'theophylline':
            new_cmpnd_name = '2-theophylline'
        elif cmpnd_name == '1_methylxanthine':
            new_cmpnd_name = '3-1_methylxanthine'
        elif cmpnd_name == '3_methylxanthine':
            new_cmpnd_name = '4-3_methylxanthine'
        elif cmpnd_name == 'hypoxanthine':
            new_cmpnd_name = '5-hypoxanthine'
        elif cmpnd_name == 'xanthine':
            new_cmpnd_name = '6-xanthine'
        elif cmpnd_name == 'caffeine':
            new_cmpnd_name = '7-caffeine'
        
        if row['acceptance_final'] == True:
            cmpnd_list_acceptance_true.append(new_cmpnd_name)
            rep_list_acceptance_true.append(new_rep_name)
        else:
            cmpnd_list_acceptance_false.append(new_cmpnd_name)
            rep_list_acceptance_false.append(new_rep_name)
    
    accept_file_name = './accept_reject_files/' + out_name + '_accept.txt'
    with open(accept_file_name, 'w') as file:
        for cmpnd, rep in zip(cmpnd_list_acceptance_true, rep_list_acceptance_true):
            entry = os.path.join(cmpnd,dir_name,rep)
            file.write(f"{entry}\t")
            
    reject_file_name = './accept_reject_files/' + out_name + '_reject.txt'
    with open(reject_file_name, 'w') as file:
        for cmpnd, rep in zip(cmpnd_list_acceptance_false, rep_list_acceptance_false):
            entry = os.path.join(cmpnd,dir_name,rep)
            file.write(f"{entry}\t")

# This function takes in a pandas DataFrame, `df`, and an offset value as input
def get_mean_std(df, offset):
    # A list of compound names to extract
    cmpnd_list = [ 'theophylline', '3_methylxanthine', 'xanthine', '1_methylxanthine', 'hypoxanthine', 'caffeine' ]
    # Extract a subset of the DataFrame where the 'acceptance_final' column is True and only include the 'ΔG(total)' column
    df2 = df.loc[df['acceptance_final'] == True, 'ΔG(total)']
    # Create empty arrays to store the mean and standard deviation of each compound's measurements
    mean_ = np.zeros(6)
    std_ = np.zeros(6)
    # iterate through 6 compunds
    for i in range(6):
        # chekc if we have 3 accepted replicas and print the Error message otherwise
        if df2.loc[cmpnd_list[i]].size != 3:
            print('Error, number of accpeted reps is not three in: ', cmpnd_list[i])
            mean_[i] = df2.loc[cmpnd_list[i]].mean()
            std_[i] = df2.loc[cmpnd_list[i]].std()
        else:
            mean_[i] = df2.loc[cmpnd_list[i]].mean()
            std_[i] = df2.loc[cmpnd_list[i]].std()

    # add the offset value to be added to all the averaged values (non-zero only for bb colvar restrains)
    mean_ += offset    
    return mean_, std_
    
# =============================================================================
# STATS FUNCTIONS
# =============================================================================

# def r2(data):
#     x, y = data.T
#     slope, intercept, r_value, p_value, stderr = scipy.stats.linregress(x, y)
#     return r_value**2


# def slope(data):
#     x, y = data.T
#     slope, intercept, r_value, p_value, stderr = scipy.stats.linregress(x, y)
#     return slope


# def me(data):
#     x, y = data.T
#     error = np.array(x) - np.array(y)
#     return error.mean()


def mae(data):
    x, y = data.T
    error = np.abs(np.array(x) - np.array(y))
    return error.mean()


def rmse(data):
    x, y = data.T
    error = np.array(x) - np.array(y)
    rmse = np.sqrt((error**2).mean())
    return rmse

def pearsons_r(data):
    x, y = data.T
    slope, intercept, r_value, p_value, stderr = scipy.stats.linregress(x, y)
    return r_value

def spearmans_rho(data):
    x, y = data.T
    spearmans_r, p_value = scipy.stats.spearmanr(x, y)
    return spearmans_r

def kendall_tau(data):
    x, y = data.T
    correlation, p_value = scipy.stats.kendalltau(x, y)
    return correlation


def compute_bootstrap_statistics(samples, stats_funcs, percentile=0.95, n_bootstrap_samples=10000):
    """Compute bootstrap confidence interval for the given statistics functions."""
    # Handle case where only a single function is passed.
    #print("SAMPLES:\n", samples)

    try:
        len(stats_funcs)
    except TypeError:
        stats_funcs = [stats_funcs]

    # Compute mean statistics.
    statistics = [stats_func(samples) for stats_func in stats_funcs]

    # Generate bootstrap statistics.
    bootstrap_samples_statistics = np.zeros((len(statistics), n_bootstrap_samples))
    for bootstrap_sample_idx in range(n_bootstrap_samples):
        samples_indices = np.random.randint(low=0, high=len(samples), size=len(samples))
        for stats_func_idx, stats_func in enumerate(stats_funcs):
            bootstrap_samples_statistics[stats_func_idx][bootstrap_sample_idx] = stats_func(samples[samples_indices])

    # Compute confidence intervals.
    percentile_index = int(np.floor(n_bootstrap_samples * (1 - percentile) / 2)) - 1
    bootstrap_statistics = []
    for stats_func_idx, samples_statistics in enumerate(bootstrap_samples_statistics):
        samples_statistics.sort()
        stat_lower_percentile = samples_statistics[percentile_index]
        stat_higher_percentile = samples_statistics[-percentile_index+1]
        confidence_interval = (stat_lower_percentile, stat_higher_percentile)
        bootstrap_statistics.append([statistics[stats_func_idx], confidence_interval, samples_statistics])

    return bootstrap_statistics

In [3]:
# Define output directory path
DIRECTORY_PATH = './performance_with_CI/'
FILE_BASE_NAME = 'performance_stats_with_CI'

In [ ]:
# Collect the statistics records for the DataFrames.
statistics_csv = []
statistics_latex = []
statistics_plot = []

#stats_funcs = collections.OrderedDict([ ('R-squared', r2), ['MAE', mae] ])
# Configuration: statistics to compute.
stats_funcs = collections.OrderedDict([
    ('MAE', mae),
    ('RMSE', rmse),
    ("pearsons_r", pearsons_r),
    ('kendall_tau', kendall_tau),
    ("spearmans_rho", spearmans_rho),
])
ordering_functions = {
    'kendall_tau': lambda x: -x
}
latex_header_conversions = {
    'MAE': 'MAE',
    'RMSE': 'RMSE',
    'pearsons_r': "Pearson's r",
    'kendall_tau': '$\\tau$',
    'spearmans_rho': "Spearman's Rho"
}

stats_names = list(stats_funcs.keys())
ci_suffixes = ('', '_lower_bound', '_upper_bound')

# Create lists of stats functions to pass to compute_bootstrap_statistics.
stats_funcs_names, stats_funcs = zip(*stats_funcs.items())

In [4]:
# Create experimental vs predicted value dataframe
# First column is experimental value and second column is predicted value and each row is a different system

###########    ['theo','mth3','xan','mth1','hyp','caf'])
exp_binding_free_energy_values = np.array([-8.86, -7.77, -6.91, -6.88, -5.88, -3.35])

In [5]:
file_dict = {
    #                                  salt cond.   # Mg   water.  lig ff    windows         BB rest. offset         title                     Dir name                       Condition ID                    
    '55_NaCl_0Mg_TP3_GAF_40_1ns_unres':['55 NaCl',     '0', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n55 mM NaCl & 0 Mg$^{2+}$",   '4-55NaCl/1-40winCmplx_30winLig', '1'],
    '55_NaCl_2Mg_TP3_GAF_40_1ns_unres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n55 mM NaCl & 2 Mg$^{2+}$",   '3-55NaCl_Mg/1-40winCmplx_30winLig', '2'],
    '55_NaCl_3Mg_TP3_GAF_40_1ns_unres':['55 NaCl',     '3', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n55 mM NaCl & 3 Mg$^{2+}$",   '1-55NaCl_3Mg/1-40winCmplx_30winLig', '3'],
    #
    '55_NaCl_0Mg_TP3_GAF_40_1ns_BBres':['55 NaCl',     '0', 'TIP3P', 'GAFF2', '40 win, 1 ns/win','Yes', 9.4, "55 mM NaCl & 0 Mg$^{2+}$\nw/ backbone restraints contribution", '9-5NaCl_bb_colvar/1-40winCmplx_30winLig', '4'],
    '55_NaCl_2Mg_TP3_GAF_40_1ns_BBres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '40 win, 1 ns/win','Yes', 9.7, "55 mM NaCl & 2 Mg$^{2+}$\nw/ backbone restraints contribution", '8-5NaCl_Mg_bb_colvar/1-40winCmplx_30winLig', '5'],
    '55_NaCl_3Mg_TP3_GAF_40_1ns_BBres':['55 NaCl',     '3', 'TIP3P', 'GAFF2', '40 win, 1 ns/win','Yes', 10,  "55 mM NaCl & 3 Mg$^{2+}$\nw/ backbone restraints contribution", '4-55NaCl_3Mg_bb_colvar/1-40winCmplx_30winLig', '6'],
    #
    '55_NaCl_2Mg_OPC_GAF_40_1ns_unres':['55 NaCl',     '2', 'OPC'  , 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "55 mM NaCl & 2 Mg$^{2+}$\nw/ OPC water", '3-55NaCl_Mg/6-opc_40winCmplx_30winLig', '7'],
    '55_NaCl_2Mg_TP3_OFF_40_1ns_unres':['55 NaCl',     '2', 'TIP3P', 'OpenFF','40 win, 1 ns/win', 'No', 0,   "55 mM NaCl & 2 Mg$^{2+}$\nw/ OpenFF", '3-55NaCl_Mg/2-OpenFF_40winCmplx_30winLig', '8'], # Latest addition   
    '55_NaCl_3Mg_TP3_OFF_40_1ns_unres':['55 NaCl',     '3', 'TIP3P', 'OpenFF','40 win, 1 ns/win', 'No', 0,   "55 mM NaCl & 3 Mg$^{2+}$\nw/ OpenFF", '1-55NaCl_3Mg/2-OpenFF_40winCmplx_30winLig', '9'],
    #
    '55_KCl_2Mg_TP3_GAF_40_1ns_unres': ['55 KCl',      '2', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n55 mM KCl & 2 Mg$^{2+}$",   '10-55KCl_Mg/1-40winCmplx_30winLig', '10'],
    '55_KCl_3Mg_TP3_GAF_40_1ns_unres': ['55 KCl',      '3', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n55 mM KCl & 3 Mg$^{2+}$",   '3-55KCl_3Mg/1-40winCmplx_30winLig', '11'],
    '150_KCl_2Mg_TP3_GAF_40_1ns_unres':['150 KCl',     '3', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\n150 mM KCl & 3 Mg$^{2+}$",   '10-55KCl_Mg/1-40winCmplx_30winLig', '12'],
    '0_NaCl_3Mg_TP3_GAF_40_1ns_unres': ['Neutralized', '3', 'TIP3P', 'GAFF2', '40 win, 1 ns/win', 'No', 0,   "\nNeutralized & 3 Mg$^{2+}$", '2-Neut_3Mg/1-40winCmplx_30winLig', '13'],
    #
    '55_NaCl_2Mg_TP3_GAF_80_1ns_unres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '80 win, 1 ns/win','No', 0,  "55 mM NaCl & 2 Mg$^{2+}$\n80 win, 1 ns/win", '3-55NaCl_Mg/2-80winCmplx', '14'],
    '55_NaCl_2Mg_TP3_GAF_40_2ns_unres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '40 win, 2 ns/win','No', 0,  "55 mM NaCl & 2 Mg$^{2+}$\n40 win, 2 ns/win", '3-55NaCl_Mg/5-40winCmplx_2ns', '15'],
    '55_NaCl_2Mg_TP3_GAF_80_1ns_BBres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '80 win, 1 ns/win','Yes', 9.7,  "55 mM NaCl & 2 Mg$^{2+}$\nw/ backbone restraints contribution\n80 win, 1 ns/win", '8-5NaCl_Mg_bb_colvar/4-80winCmplx', '16'],
    '55_NaCl_2Mg_TP3_GAF_40_2ns_BBres':['55 NaCl',     '2', 'TIP3P', 'GAFF2', '40 win, 2 ns/win','Yes', 9.7,  "55 mM NaCl & 2 Mg$^{2+}$\nw/ backbone restraints contribution\n40 win, 2 ns/win", '8-5NaCl_Mg_bb_colvar/5-40winCmplx_2ns', '17'],
    '55_NaCl_0Mg_TP3_GAF_80_1ns_BBres':['55 NaCl',     '0', 'TIP3P', 'GAFF2', '80 win, 1 ns/win','Yes', 9.4,  "55 mM NaCl & 0 Mg$^{2+}$\nw/ backbone restraints contribution\n80 win, 1 ns/win", '9-5NaCl_bb_colvar/4-80winCmplx', '18'],
    '55_NaCl_0Mg_TP3_GAF_40_2ns_BBres':['55 NaCl',     '0', 'TIP3P', 'GAFF2', '40 win, 2 ns/win','Yes', 9.4,  "55 mM NaCl & 0 Mg$^{2+}$\nw/ backbone restraints contribution\n40 win, 2 ns/win", '9-5NaCl_bb_colvar/3-40winCmplx_2ns', '19'],
}

In [6]:
# Iterate through conditions
for key in file_dict.keys():

    path = os.path.join('./BFE_failed_rep/',key + '.txt')
    
    cond_name = key
    print(cond_name)
    
    cond_id = file_dict[key][9]
    print(cond_id)
    
    df = read_file(path)
    get_failure(df)
    # Select accepted replicates only
    #df_accept = df[df['acceptance_final'] == True]
    #display(df_accept)
    write_accept_reject(df, file_dict[key][8],key)
    # Calculate mean predicted values and standard deviation for each system
    mean, std = get_mean_std(df, file_dict[key][6]) # second number is offset value (non-zero only for colvar bb rst.)
    print(mean)
    print(std)
    break

    #file_dict = plotting(mean, std, file_dict, key, cond_id)
    

55_NaCl_0Mg_TP3_GAF_40_1ns_unres
1
[ -8.86468311 -10.17995765  -7.05708094  -8.92615367  -2.20755616
  -3.60227173]
[3.57432172 2.75868451 5.69031628 2.42270137 0.56347642 3.69907004]


In [7]:
# Create horizontal stacking of predicted and experimental values
data = np.vstack((np.array(mean), exp_binding_free_energy_values))
print(data.shape)
data

(2, 6)


array([[ -8.86468311, -10.17995765,  -7.05708094,  -8.92615367,
         -2.20755616,  -3.60227173],
       [ -8.86      ,  -7.77      ,  -6.91      ,  -6.88      ,
         -5.88      ,  -3.35      ]])

In [9]:

# We will bootstrap over systems with replacement 10 000 times.
data = pd.DataFrame(data).transpose()
samples = data.to_numpy()
bootstrap_statistics = compute_bootstrap_statistics(samples, stats_funcs, n_bootstrap_samples=1000) #10000 

# Return statistics as dict preserving the order.
bootstrap_statistics_dict = collections.OrderedDict((stats_funcs_names[i], bootstrap_statistics[i]) for i in range(len(stats_funcs)))
bootstrap_statistics_dict


OrderedDict([('MAE',
              [1.4220984889444666,
               (0.4274577446460726, 2.603420627115298),
               array([0.07588203, 0.08721265, 0.11094562, 0.1346786 , 0.1346786 ,
                      0.1346786 , 0.1346786 , 0.1346786 , 0.14087977, 0.17594337,
                      0.17594337, 0.18214454, 0.19347516, 0.19967634, 0.3449282 ,
                      0.36866118, 0.38619297, 0.39239415, 0.39239415, 0.39239415,
                      0.40992595, 0.40992595, 0.40992595, 0.41612712, 0.42745774,
                      0.42745774, 0.42745774, 0.42929517, 0.43365892, 0.43365892,
                      0.43365892, 0.43365892, 0.43365892, 0.44682697, 0.45119072,
                      0.45302814, 0.45739189, 0.45739189, 0.45739189, 0.45739189,
                      0.46872251, 0.47055994, 0.47055994, 0.47492369, 0.47492369,
                      0.47492369, 0.47492369, 0.47492369, 0.47676112, 0.48809174,
                      0.49245549, 0.49429291, 0.49429291, 0.49429291

In [10]:
# Organize data to construct CSV and PDF versions of statistics tables
record_csv = {}
record_latex = {}
for stats_name, (stats, (lower_bound, upper_bound), bootstrap_samples) in bootstrap_statistics_dict.items():
    # For CSV and JSON we put confidence interval in separate columns.
    for suffix, info in zip(ci_suffixes, [stats, lower_bound, upper_bound]):
        record_csv[stats_name + suffix] = info

    # For the PDF, print bootstrap CI in the same column.
    stats_name_latex = latex_header_conversions.get(stats_name, stats_name)
    record_latex[stats_name_latex] = '{:.2f} [{:.2f}, {:.2f}]'.format(stats, lower_bound, upper_bound)

    # For the violin plot, we need all the bootstrap statistics series.
    for bootstrap_sample in bootstrap_samples:
        statistics_plot.append(dict(ID=cond_id, name=cond_name, statistics=stats_name_latex, value=bootstrap_sample))

statistics_csv.append({'ID': cond_id, 'name': cond_name, **record_csv})
escaped_name = cond_name.replace('_', '\_')
statistics_latex.append({'ID': cond_id, 'name': escaped_name, **record_latex})

print()
print("statistics_csv:\n",statistics_csv)
print()

# Convert dictionary to Dataframe to create tables/plots easily.
statistics_csv = pd.DataFrame(statistics_csv)
display(statistics_csv)
#statistics_csv.set_index('ID', inplace=True)
statistics_latex = pd.DataFrame(statistics_latex)
statistics_plot = pd.DataFrame(statistics_plot)

# Sort by the given statistics.
sort_stat = None
if sort_stat is not None:
    statistics_csv.sort_values(by=sort_stat, inplace=True)
    statistics_latex.sort_values(by=latex_header_conversions.get(sort_stat, sort_stat),
                                    inplace=True)

# Reorder columns that were scrambled by going through a dictionaries.
stats_names_csv = [name + suffix for name in stats_names for suffix in ci_suffixes]
#print("stats_names_csv:", stats_names_csv)
stats_names_latex = [latex_header_conversions.get(name, name) for name in stats_names]
#print("stats_names_latex:", stats_names_latex)
statistics_csv = statistics_csv[['name'] + stats_names_csv ]
statistics_latex = statistics_latex[['ID', 'name'] + stats_names_latex ] 

# Create CSV and JSON tables (correct LaTex syntax in column names).
os.makedirs(DIRECTORY_PATH)
file_base_path = os.path.join(DIRECTORY_PATH, FILE_BASE_NAME)
with open(file_base_path + '.csv', 'w') as f:
    statistics_csv.to_csv(f)
with open(file_base_path + '.json', 'w') as f:
    statistics_csv.to_json(f, orient='index')

# Create LaTex table.
latex_directory_path = os.path.join(DIRECTORY_PATH, FILE_BASE_NAME + 'LaTex')
os.makedirs(latex_directory_path, exist_ok=True)
with open(os.path.join(latex_directory_path, FILE_BASE_NAME + '.tex'), 'w') as f:
    f.write('\\documentclass{article}\n'
            '\\usepackage[a4paper,margin=0.005in,tmargin=0.5in,lmargin=0.5in,rmargin=0.5in,landscape]{geometry}\n'
            '\\usepackage{booktabs}\n'
            '\\usepackage{longtable}\n'
            '\\pagenumbering{gobble}\n'
            '\\begin{document}\n'
            '\\begin{center}\n'
            '\\scriptsize\n')
    statistics_latex.to_latex(f, column_format='|ccccccccc|', escape=False, index=False, longtable=True)
    f.write('\end{center}\n'
            '\nNotes\n\n'
            '- Mean and 95\% confidence intervals of performance statistics were calculated by bootstrapping with 1000 samples.\n\n'
            '\end{document}\n')


statistics_csv:
 [{'ID': '1', 'name': '55_NaCl_0Mg_TP3_GAF_40_1ns_unres', 'MAE': 1.4220984889444666, 'MAE_lower_bound': 0.4274577446460726, 'MAE_upper_bound': 2.603420627115298, 'RMSE': 1.9818669722611426, 'RMSE_lower_bound': 0.8459408291517743, 'RMSE_upper_bound': 2.9004828532014453, 'pearsons_r': 0.7602795001932169, 'pearsons_r_lower_bound': 0.23250332383746375, 'pearsons_r_upper_bound': 0.9999319879427876, 'kendall_tau': 0.4666666666666666, 'kendall_tau_lower_bound': -0.23076923076923078, 'kendall_tau_upper_bound': 1.0, 'spearmans_rho': 0.6571428571428573, 'spearmans_rho_lower_bound': -0.33333333333333326, 'spearmans_rho_upper_bound': 1.0}]



,ID,name,MAE,MAE_lower_bound,MAE_upper_bound,RMSE,RMSE_lower_bound,RMSE_upper_bound,pearsons_r,pearsons_r_lower_bound,pearsons_r_upper_bound,kendall_tau,kendall_tau_lower_bound,kendall_tau_upper_bound,spearmans_rho,spearmans_rho_lower_bound,spearmans_rho_upper_bound
0,1,55_NaCl_0Mg_TP3_GAF_40_1ns_unres,1.422098,0.427458,2.603421,1.981867,0.845941,2.900483,0.76028,0.232503,0.999932,0.466667,-0.230769,1.0,0.657143,-0.333333,1.0
